In [325]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_auc_score

%run ../src/Munger.py
%run ../src/Models.py
%run ../src/utils.py

In [6]:
# load dataset
numer_ai = pd.read_csv('../data/numerai_datasets/numerai_training_data.csv')
test = pd.read_csv('../data/numerai_datasets/numerai_tournament_data.csv')

In [7]:
# split into training and validation sets as per the competition instructions
train = numer_ai[numer_ai.validation == 0]
validation = numer_ai[numer_ai.validation == 1]

In [326]:
munger = Munger(train, validation, test)

In [327]:
## remove correlated features
munger.remove_correlated_features()

In [328]:
## label encoding a categorical feature
munger.label_encoding()

C:\Users\Abhishek\Anaconda2\lib\site-packages\pandas\core\indexing.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Abhishek\Anaconda2\lib\site-packages\pandas\core\indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [303]:
## one hot encoding categorical feature
munger.one_hot_encoding()

In [329]:
X_train = munger.X
y_train = munger.y

X_validation = munger.X_validation
y_validation = munger.y_validation

X_test = munger.X_test

In [330]:
model = Models()
clf = model.logistic_regression_model()

In [331]:
## 5-fold cross validation
mean_score, mean_std = eval_models([clf], X_train, y_train, 
                                   X_validation, y_validation)

IndexError: positional indexers are out-of-bounds

In [308]:
# fit a model
clf.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0))])

In [310]:
## predictions
predsTrain = clf.predict_proba(X_train)[:, 1]
predsValidation = clf.predict_proba(X_val)[:, 1]
predsTest = clf.predict_proba(X_t)[:, 1]

In [311]:
## check to see how this dummy classifier performs
print 'ROC AUC Score on the training examples %f ' %(roc_auc_score(y_train, predsTrain))
print 'ROC AUC Score on the validation examples %f ' %(roc_auc_score(y_val, predsValidation))
print 'ROC AUC Score on the validation examples %f ' %(roc_auc_score(y_t, predsTest))

ROC AUC Score on the training examples 0.536529 
ROC AUC Score on the validation examples 0.527965 
ROC AUC Score on the validation examples 0.539497 


In [312]:
# create full dataset
munger.concatenate_train_validation()

In [313]:
# full dataset
X_full = munger.X_full
y_full = munger.y_full

In [314]:
## fit on whole dataset
clf.fit(X_full, y_full)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0))])

In [315]:
predictions = clf.predict_proba(X_test)[:, 1]

In [316]:
# submission dataframe
submission_df = pd.read_csv('../data/numerai_datasets/numerai_example_predictions.csv')

In [317]:
prepare_submission(submission_df, predictions, 'logistic_reg_ohe.csv')